# 寻路、识别动物的比赛

## 识别两次动物，第一次停10秒，第二次彻底停止

引入相关模块

In [ ]:
import torchvision
import torch
import time
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from jetbot import bgr8_to_jpeg
from jetbot import Robot
from jetbot import Camera
from jetbot import ObjectDetector

初始化相机，方便后面的调用

In [ ]:
camera = Camera.instance(width=224, height=224,fps=10)

获取cuda设备的引用，方便后面使用

In [ ]:
device = torch.device('cuda')

加载识别动物的模型

In [ ]:
model_object = ObjectDetector('ssd_mobilenet_v2_coco.engine')

初始化小车的实例

In [ ]:
robot = Robot()

导入resnet18的网络模型，pretrained=False的意思是只用网络结构，不需要预训练网络模型的参数做初始化

In [ ]:
model = torchvision.models.resnet18(pretrained=False)

对输入数据进行线性变换，输入是512维，输出是2维

In [ ]:
model.fc = torch.nn.Linear(512, 2)

加载模型huangsexian_304zhang_50epoch.pth

In [ ]:
model.load_state_dict(torch.load('huangsexian_304zhang_50epoch.pth'))#黄色中间线模型
#model.load_state_dict(torch.load('huangsexian_200zhang.pth'))#黄色中间线模型

将模型放到device上，就是前面定义的对cuda的引用

In [ ]:
model = model.to(device)

对模型求值

In [ ]:
model = model.eval().half()

定义均值和方差，方便后面使用

In [2]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

# objectId是动物对应的索引

16是鸟、17是猫、18是狗、19是马、20是羊、21是牛、22是大象、23是熊、24是斑马、25是长颈鹿

如果找到模型通过相机检测到对应的动物就返回True，没有检测到就返回False

In [3]:
def find_object(objectId):
    image = camera.value
    image = cv2.resize(image,(300,300))
    #图像是300x300的可以识别，224x224的识别错误
    detections = model_object(image)
    matching_detections = [d for d in detections[0] if d['label'] == int(objectId)]
    
    if(len(matching_detections) != 0):
        return True
    else:
        return False

预处理图片

In [4]:
def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

定义滑块来控制JetBot，对不同的值进行控制
1. 速度控制（speed_gain_slider）：开始增加``speed_gain_slider``来启动JetBot
2. 转向增益控制（steering_gain_sloder）：如果您看到JetBot摇摆不定，则需要减小``steering_gain_slider``直到平稳为止
3. 转向偏差控制（steering_bias_slider）：如果看到JetBot偏向轨道的最右侧或最左侧，则应控制此滑块，直到JetBot开始沿中心线或轨道行进。这说明了电机偏差以及相机偏移

In [ ]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.47, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=0.2, step=0.001, value=0.09, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.2, step=0.001, value=0.03, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.1, max=0.1, step=0.001, value=0.01, description='steering bias')

将定义的控制调节的滑块显示出来

In [ ]:
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

stopped初始值为0，第一次停止将stopped设置为1，第二次停止将stopped设置为2，，stopped为2时不再给马达赋值

road_or_animal用来判断哪些图片是用来寻路的，哪些图片是用来识别动物的。
之所以添加这个限制是因为如果所有的图片都运行一遍寻路且都运行一遍识别动物时，nano的计算能力是不够的，会导致运行效果极差



In [7]:
angle = 0.0
angle_last = 0.0
stopped = 0
road_or_animal = 0

def execute(image):
    global angle, angle_last
    global stopped
    global road_or_animal
    road_or_animal += 1
   
    if stopped < 2:#寻路，发现目标就停下
        if road_or_animal%4 != 0:#3帧寻路1帧识别，交叉运行
            xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
            x = xy[0]
            y = (0.5 - xy[1]) / 2.0
            speed_slider_value = speed_gain_slider.value
            angle = np.arctan2(x, y)
            pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
            angle_last = angle
            steering_slider_value = pid + steering_bias_slider.value
            my_value0 = 0.5
            my_value = 0.5
            robot.left_motor.value = max(min(speed_slider_value*my_value0 + steering_slider_value*my_value, 1), 0.0)
            robot.right_motor.value = max(min(speed_slider_value*my_value0 - steering_slider_value*my_value, 1), 0.0)
                
        else:
            if stopped == 0:#第一次停止
                if find_object(objectId = 18) == True:
                    print('小车停一下')
                    #16是鸟、17是猫、18是狗、19是马、20是羊、21是牛、22是大象、23是熊、24是斑马、25是长颈鹿
                    stopped = 1
                    robot.left_motor.value = 0
                    robot.right_motor.value = 0
                    time.sleep(10)
            else:
                if stopped == 1:
                    if find_object(objectId = 18) == True:#第二次停止
                        print('彻底停止了')
                        stopped = 2#速度设置为0后跳出执行
                        robot.left_motor.value = 0
                        robot.right_motor.value = 0

count是用来设置小车在调用摄像头之前的预热启动的帧数，此时小车在不调用摄像头的情况下运行

运行count帧数后才开始调用摄像头寻路

In [8]:
count = 0
def switch(change):
    global count
    count += 1
    if count <= 6:
        robot.left_motor.value = 0.29
        robot.right_motor.value = 0.29
        return 0
    image = change['new']
    execute(image)

当相机的值改变时，observe方法的回调函数也会被调用，也就是相机的图片有变动就会调用switch方法

In [ ]:
camera.observe(switch, names='value')

接触相机对回调函数switch的绑定，此后相机的改变不再调用switch函数

In [11]:
camera.unobserve(switch, names='value')

释放camera对相机的引用，释放后不能通过camera对相机进行调用

In [12]:
camera.stop()

稍微等一下再停止小车，让小车有时间处理最后的图片

In [13]:
time.sleep(0.1)
robot.stop()